In [ ]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image

In [ ]:
path = '../input/plant-pathology-2021-fgvc8'
train_dir = '../input/plant-pathology-2021-fgvc8/train_images'
test_dir = '../input/plant-pathology-2021-fgvc8/test_images' 

df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
df.labels.value_counts()

In [ ]:
df['labels'].unique()

In [ ]:
df['labels'] = df['labels'].astype(str) 

In [ ]:
'''
def plot_examples(label):
    fig, ax = plt.subplots(1, 2, figsize=(25, 15))
    ax = ax.ravel()
    for i in range(2):
        idx = df[df['labels']==label].index[i]
        image = cv2.imread(train_dir+df.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])

for labels in list(df['labels'].unique()):
    plot_examples(labels)'''

In [ ]:
train_datagen  = ImageDataGenerator(rotation_range = 180,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split = 0.2)
test_datagen  = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

train_generator = train_datagen.flow_from_dataframe(dataframe = df,
                                                   directory = train_dir,
                                                   target_size = (150,150),
                                                   x_col = 'image',
                                                   y_col = 'labels',
                                                   batch_size = 256,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                   subset = 'training')

test_generator = test_datagen.flow_from_dataframe(dataframe = df,
                                                 directory = train_dir,
                                                 target_size = (150,150),
                                                 x_col = 'image',
                                                 y_col = 'labels',
                                                 batch_size = 256,
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 subset = 'validation')


In [ ]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu',padding='same',input_shape=[150,150,3]))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(16,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(8,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(8,(3,3),activation='relu',padding='same'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(12,activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam
epochs = 10
Batch_size = 64
optimizer = Adam(lr = 0.001)
model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])

model.summary()

In [ ]:
#history = model.fit_generator(train_generator,epochs = epochs,validation_data = test_generator)
history = model.fit(train_generator,epochs = 2,batch_size=64,validation_data = test_generator)

#prediction = model.predict(test_generator)

In [ ]:
prediction = model.predict(test_generator)

In [ ]:
prediction = np.argmax(prediction,axis=1)

In [ ]:
dictionary = {1:'healthy',2: 'scab frog_eye_leaf_spot complex',3: 'scab',4: 'complex',5:'rust', 6:'frog_eye_leaf_spot', 
 7:'powdery_mildew',8:'scab frog_eye_leaf_spot', 9:'frog_eye_leaf_spot complex',10: 'rust frog_eye_leaf_spot', 
 11: 'powdery_mildew complex',12: 'rust complex'}

In [ ]:
classes = []
prediction = []
for i in range(0,len(prediction)):
    temp = prediction[i]
    ans = dictionary[temp]
    classes.append(ans)
    
#print(classes)

In [ ]:
test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv') 
print(test.shape)


test_dir = '../input/plant-pathology-2021-fgvc8/test_images'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

In [ ]:
datagen  = ImageDataGenerator(rotation_range = 180,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   zoom_range = 0.2,
                                   validation_split = 0.2)

test_set = datagen.flow_from_dataframe(
    dataframe= test_df,
    directory = test_dir,
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (150,150),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

print(test_set[0].shape)

In [ ]:
print(test_set[0].shape)

In [ ]:
preds = model.predict(test_set)

preds

In [ ]:
final = []
index = []
for i in range(0,len(preds)):
    temp = preds[i]
    l = []
    for j in range(0,len(temp)):
        if j==0 and temp[j]>0.27:
            l.append(j+1)
            break
        if temp[j]>0.14 and j!=0:     
            l.append(j+1)
    index.append(l)
    
print(index)

'''labels = []
for i in range(0,index):
    temp = index[i]
'''


'''    
test_df['labels'] = classes
test_df.to_csv('submission.csv', index=False) 
test_df.head()'''

In [ ]:
labels_out = []
for i in range(0,len(index)):
    t = ''
    for j in range(0,len(index[i])):
        t = t + dictionary[index[i][j]] + ' '
    if len(t)==0:
        t = 'healthy' 
    labels_out.append(t)

print(labels_out)

In [ ]:
from collections import Counter
  
def remov_duplicates(input):
  
    input = input.split(" ")
  
    for i in range(0, len(input)):
        input[i] = "".join(input[i])
  
    UniqW = Counter(input)
  
    s = " ".join(UniqW.keys())
    return s

In [ ]:
submit = []

for i in range(0,len(labels_out)):
    ans = remov_duplicates(labels_out[i])
    submit.append(ans)
    
print(submit)

In [ ]:
test_df['labels'] = submit
test_df.to_csv('submission.csv', index=False)
test_df.head()

In [ ]:
df = pd.read_csv("./submission.csv")
df.head()